# import

In [1]:
import pandas as pd
import numpy as np
import time
from sklearn.metrics import roc_auc_score
from sklearn.feature_selection import mutual_info_classif
import datetime
import pickle
import scipy.sparse as ss
import logging
LOG_FORMAT = "%(asctime)s - %(levelname)s - %(message)s"
logging.basicConfig(level=logging.INFO, format=LOG_FORMAT)
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
os.environ['NUMEXPR_MAX_THREADS'] = '32'
import seaborn as sns

import IPython.display as ipd
import copy
import random
# from pandarallel import pandarallel
# Initialization
# pandarallel.initialize(progress_bar=True)
# df.parallel_apply(func)
from tqdm import tqdm_notebook as tqdm

from gensim.models.word2vec import Word2Vec 
import lightgbm as lgb
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold,KFold

2020-07-23 09:30:01,503 - WARNING - unable to import 'smart_open.gcs', disabling that module


# Read

In [2]:
var_dir = '../../var'
my_var_dir = '../../var/hyr'
fjw_var_dir = '../../var/fjw'

In [3]:
df_train_click = pd.read_csv('%s/data/train_semi_final/click_log.csv' % var_dir)
df_train_ad = pd.read_csv('%s/data/train_semi_final/ad.csv' % var_dir)

df_test_click = pd.read_csv('%s/data/test/click_log.csv' % var_dir)
df_test_ad = pd.read_csv('%s/data/test/ad.csv' % var_dir)

In [4]:
pre_df_train_click = pd.read_csv('%s/data/train_preliminary/click_log.csv' % var_dir)
pre_df_train_ad = pd.read_csv('%s/data/train_preliminary/ad.csv' % var_dir)
df_train_click = pd.concat([pre_df_train_click, df_train_click])
df_train_ad = pd.concat([pre_df_train_ad, df_train_ad])


In [5]:
df_train_user = pd.read_csv('%s/data/train_semi_final/user.csv'% var_dir)
pre_df_train_user = pd.read_csv('%s/data/train_preliminary/user.csv'% var_dir)
df_train_user = pd.concat([pre_df_train_user, df_train_user])

# preprocess

## dump

### 存整个

In [6]:
df_train_click['type'] = 'train'
df_test_click['type'] = 'test'
df_click = pd.concat([df_train_click, df_test_click])
df_ad = pd.concat([df_train_ad, df_test_ad])
df_ad = df_ad.drop_duplicates()
df_click = df_click.sort_values(by = ['time'])

In [7]:
gby_user_id = df_click[['user_id', 'creative_id']].groupby('user_id')

#se_user_creative_id
mp_user_creative = {}
for user, df_item in tqdm(gby_user_id):
    mp_user_creative[user] = list(df_item['creative_id'])
se_user_creative_id = pd.Series(mp_user_creative)
pickle.dump(se_user_creative_id, open('%s/se_user_creative_id.pickle' % my_var_dir, 'wb'))

#se_user_time, se_user_click_times
gby_user_id = df_click[['user_id', 'time', 'click_times']].groupby('user_id')
mp_user_time = {}
mp_user_click_times = {}
for user, df_item in tqdm(gby_user_id):
    mp_user_time[user] = list(df_item['time'])
    mp_user_click_times[user] = list(df_item['click_times'])
se_user_time = pd.Series(mp_user_time)
se_user_click_times = pd.Series(mp_user_click_times)
pickle.dump(se_user_time, open('%s/se_user_time.pickle' % my_var_dir, 'wb'))
pickle.dump(se_user_click_times, open('%s/se_user_click_times.pickle' % my_var_dir, 'wb'))


df_ad.index = df_ad.creative_id
for feature in ['ad_id', 'product_id', 'product_category', 'advertiser_id', 'industry']:
    pickle.dump(df_ad[feature], open('%s/se_creative_id_%s.pickle' % ( my_var_dir, feature), 'wb'))

/home/huangweilin/anaconda3/envs/fjw/lib/python3.6/site-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


/home/huangweilin/anaconda3/envs/fjw/lib/python3.6/site-packages/ipykernel_launcher.py:14: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [8]:
logging.info('load')

se_user_creative_id = pickle.load(open('%s/se_user_creative_id.pickle' % my_var_dir, 'rb'))
mp_creative_other = {
    'ad_id' : pickle.load(open('%s/se_creative_id_ad_id.pickle'% my_var_dir, 'rb')),
    'advertiser_id' : pickle.load(open('%s/se_creative_id_advertiser_id.pickle'% my_var_dir, 'rb')),
    'industry' : pickle.load(open('%s/se_creative_id_industry.pickle'% my_var_dir, 'rb')),
    'product_category' : pickle.load(open('%s/se_creative_id_product_category.pickle'% my_var_dir, 'rb')),
    'product_id' : pickle.load(open('%s/se_creative_id_product_id.pickle'% my_var_dir, 'rb')),
}
logging.info('finish load')


se_user_creative_id_origin = None
def sub_creative_id():
    global se_user_creative_id, se_user_creative_id_origin
    assert se_user_creative_id.shape[0] == 4000000
    se_user_creative_id_origin = se_user_creative_id
    se_user_creative_id = se_user_creative_id.iloc[:1024 * 5]
    
def full_creative_id():
    global se_user_creative_id, se_user_creative_id_origin
    se_user_creative_id = se_user_creative_id_origin
    
def get_feature_se(name):
    if(name == "creative_id"):
        return se_user_creative_id
    
    mp_user_other = {}
    for user in tqdm(se_user_creative_id.index):
        mp_user_other[user] = list(mp_creative_other[name][se_user_creative_id[user]])
    se_user_other = pd.Series(mp_user_other)
    return se_user_other

2020-07-23 09:30:08,624 - INFO - load
2020-07-23 09:30:08,632 - INFO - finish load


In [9]:
mp_creative_other['product_id'] = mp_creative_other['product_id'].replace("\\N", 0)
mp_creative_other['industry'] = mp_creative_other['industry'].replace("\\N", 0)
mp_creative_other['product_id'] = mp_creative_other['product_id'].apply(lambda x : int(x))
mp_creative_other['industry'] = mp_creative_other['industry'].apply(lambda x : int(x))

In [10]:
for feature in ['ad_id', 'product_id', 'product_category', 'advertiser_id', 'industry']:
    se = get_feature_se(feature)
    pickle.dump(se, open('%s/se_user_%s.pickle' % (my_var_dir, feature), 'wb'))

/home/huangweilin/anaconda3/envs/fjw/lib/python3.6/site-packages/ipykernel_launcher.py:30: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


## load

In [11]:
logging.info('load')
mp_id_se = {
    'creative_id' : pickle.load(open('%s/se_user_creative_id.pickle' % my_var_dir, 'rb')),
    'ad_id' : pickle.load(open('%s/se_user_ad_id.pickle' % my_var_dir, 'rb')),
    'advertiser_id' : pickle.load(open('%s/se_user_advertiser_id.pickle'% my_var_dir, 'rb')),
    'industry' : pickle.load(open('%s/se_user_industry.pickle'% my_var_dir, 'rb')),
    'product_category' : pickle.load(open('%s/se_user_product_category.pickle' % my_var_dir, 'rb')),
    'product_id' : pickle.load(open('%s/se_user_product_id.pickle' % my_var_dir, 'rb')),
    'time' : pickle.load(open('%s/se_user_product_id.pickle' % my_var_dir, 'rb')),
    'click_times' : pickle.load(open('%s/se_user_click_times.pickle' % my_var_dir, 'rb')),
}
logging.info('finish load')

2020-07-23 09:30:10,672 - INFO - load
2020-07-23 09:30:10,688 - INFO - finish load


# merge df_user_info

In [12]:
import pickle
se_user_creative_id = pickle.load(open('%s/se_user_creative_id.pickle'%my_var_dir, 'rb'))
se_user_time = pickle.load(open('%s/se_user_time.pickle'%my_var_dir, 'rb'))
se_user_click_times = pickle.load(open('%s/se_user_click_times.pickle'%my_var_dir, 'rb'))
new_seq_list = []
for seq in se_user_click_times:
    new_seq = list(map(lambda x : x if x < 32 else 32, seq))
    new_seq_list.append(new_seq)
se_user_click_times_cut = pd.Series(new_seq_list)
se_user_click_times_cut.index = se_user_click_times.index
df_user_info = pd.DataFrame(list(zip(se_user_creative_id, se_user_time, se_user_click_times_cut)), columns = ['creative_id', 'time', 'click_time'])
df_user_info.index = se_user_creative_id.index
pickle.dump(df_user_info, open('%s/df_user_info.pickle' % my_var_dir, 'wb'))

# tfidf

In [13]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from category_encoders import *
from scipy import stats
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
import scipy
import lightgbm as lgb
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold,KFold
import pickle
import logging
LOG_FORMAT = "%(asctime)s - %(levelname)s - %(message)s"
logging.basicConfig(level=logging.INFO, format=LOG_FORMAT)
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
os.environ['NUMEXPR_MAX_THREADS'] = '32'
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm


## sequence (idf map)

In [14]:
feature_name = ['creative_id', 'ad_id', 'product_id', 'product_category', 'advertiser_id', 'industry', 'time', 'click_times']

def generate_idf_sequence(se_data):
    cross_corpus = list(se_data.apply(lambda a_list : ' '.join([str(x) for x in a_list])))
    tfidf_vectorizer = TfidfVectorizer(analyzer='word', token_pattern=r"\S+", lowercase = False)
    tfidf_spare = tfidf_vectorizer.fit_transform(cross_corpus)
    idf_dict = dict(zip(tfidf_vectorizer.get_feature_names(), tfidf_vectorizer.idf_))
    se_idf = pd.Series(idf_dict)
    ipd.display(se_idf.iloc[:10])
    pickle.dump(se_idf, open('%s/%s_idf.pickle' % (my_var_dir, column_name), 'wb'))

for column_name in feature_name:
    logging.info('start tfidf sequence %s' % column_name)
    se = mp_id_se[column_name]
    generate_idf_sequence(se)
    logging.info('finish tfidf sequence %s' % column_name)


2020-07-23 09:30:14,327 - INFO - start tfidf sequence creative_id


1000368    6.300814
1001467    6.300814
1002034    6.300814
1003301    6.300814
1003322    6.300814
1003624    6.300814
1003868    6.300814
1003974    6.300814
1004768    6.300814
100501     6.300814
dtype: float64

2020-07-23 09:30:14,399 - INFO - finish tfidf sequence creative_id
2020-07-23 09:30:14,400 - INFO - start tfidf sequence ad_id


1000071    6.300814
1000347    6.300814
1000638    6.300814
1001429    6.300814
1001456    6.300814
1001574    6.300814
1001688    6.300814
1001701    6.300814
1001746    6.300814
1001761    6.300814
dtype: float64

2020-07-23 09:30:14,466 - INFO - finish tfidf sequence ad_id
2020-07-23 09:30:14,467 - INFO - start tfidf sequence product_id


...

2020-07-23 09:30:14,552 - INFO - finish tfidf sequence industry
2020-07-23 09:30:14,553 - INFO - start tfidf sequence time


0       1.005000
100     6.300814
101     4.049522
1014    6.300814
1017    4.796737
102     5.895349
1020    6.300814
1022    6.300814
1023    5.607667
1026    5.895349
dtype: float64

2020-07-23 09:30:14,573 - INFO - finish tfidf sequence time
2020-07-23 09:30:14,574 - INFO - start tfidf sequence click_times


1    1.000000
2    1.521691
3    3.661757
4    4.429012
6    6.300814
8    5.895349
9    6.300814
dtype: float64

2020-07-23 09:30:14,588 - INFO - finish tfidf sequence click_times


## stack feature

In [15]:
def generate_tfidf(column_name):
    se = mp_id_se[column_name]
    cross_corpus = list(se.apply(lambda a_list : ' '.join(['%s_'%column_name + str(x) for x in a_list])))
    tfidf_vectorizer = TfidfVectorizer(max_features = 50000)
    tfidf_spare = tfidf_vectorizer.fit_transform(cross_corpus)
    return tfidf_spare

def generate_count_word(column_name):
    se = mp_id_se[column_name]
    cross_corpus = list(se.apply(lambda a_list : ' '.join(['%s_'%column_name + str(x) for x in a_list])))
    count_vectorizer = CountVectorizer(max_features = 50000)
    count_spare = count_vectorizer.fit_transform(cross_corpus)
    return count_spare

feature_name = ['time', 'creative_id', 'ad_id', 'product_id', 'product_category', 'advertiser_id', 'industry']
spare_m_list = []
logging.info('start cat_matrix')
for name in feature_name:
    tfidf = generate_tfidf(name)
#     count_word = generate_count_word(name)
    spare_m_list.append(tfidf)
#     spare_m_list.append(count_word)  
    logging.info('finish %s' % name)
    print(tfidf.shape)

logging.info('finish cat_matrix')

cat_matrix = scipy.sparse.hstack(spare_m_list)
cat_matrix.shape

2020-07-23 09:30:15,794 - INFO - start cat_matrix
2020-07-23 09:30:15,826 - INFO - finish time
2020-07-23 09:30:15,879 - INFO - finish creative_id
2020-07-23 09:30:15,928 - INFO - finish ad_id
2020-07-23 09:30:15,949 - INFO - finish product_id
2020-07-23 09:30:15,968 - INFO - finish product_category
2020-07-23 09:30:15,995 - INFO - finish advertiser_id
2020-07-23 09:30:16,014 - INFO - finish industry
2020-07-23 09:30:16,015 - INFO - finish cat_matrix


(400, 1261)
(400, 9984)
(400, 9909)
(400, 1261)
(400, 16)
(400, 3628)
(400, 200)


(400, 26259)

In [16]:
# df_train_user = pd.read_csv('/data/ccnth/algo/train_semi_final/user.csv')
# pre_df_train_user = pd.read_csv('/data/ccnth/algo/train_preliminary/user.csv')
# df_labels = pd.concat([pre_df_train_user, df_train_user])
df_labels = df_train_user
cat_matrix = cat_matrix.tocsr()

In [18]:
def tfidf_count_cross_validation(train_x, test_x, y, model_class, n_class):
    
    n_flod = 5
    folds = KFold(n_splits=n_flod, shuffle=True, random_state=0)
    
    if n_class > 2:
        val_score = np.zeros((train_x.shape[0], n_class))
        test_score = np.zeros((test_x.shape[0], n_class))
    else:
        val_score = np.zeros((train_x.shape[0], 1))
        test_score = np.zeros((test_x.shape[0], 1))
        
    for n_fold, (trn_idx, val_idx) in enumerate(folds.split(train_x, y)):
        logging.info('fold:%d' % n_fold)
        model = model_class()
        trn_x, trn_y = train_x[trn_idx], y[trn_idx]
        val_x, val_y = train_x[val_idx], y[val_idx]
        model.fit(trn_x, trn_y)
        
        try:
            val_ret = model.predict_proba(val_x)
            test_ret = model.predict_proba(test_x)
            if(n_class == 2):
                val_ret = val_ret[:, 1]
                test_ret = test_ret[:, 1]
        except:
            val_ret = model.decision_function(val_x)
            test_ret = model.decision_function(test_x)

        
        if(n_class == 2):
            val_ret = val_ret.reshape(-1, 1)
            test_ret = test_ret.reshape(-1, 1)

        val_score[val_idx] = val_ret
        test_score += test_ret / n_flod
        
    return val_score, test_score


model_classes = [SGDClassifier, PassiveAggressiveClassifier, BernoulliNB, MultinomialNB, LogisticRegression]
#ok : SGDClassifier, PassiveAggressiveClassifier, BernoulliNB, MultinomialNB, LogisticRegression
#not ok RidgeClassifier, DecisionTreeClassifier RandomForestClassifier ExtraTreesClassifier
# model_classes = [ SGDClassifier, PassiveAggressiveClassifier, RidgeClassifier, BernoulliNB, MultinomialNB]

train_score_list, test_score_list = [], []

n = 1
for i, model_class in enumerate(model_classes):    
    logging.info('start: %d' % i)
    val_score, test_score = \
    tfidf_count_cross_validation(cat_matrix[:int(3000000/n)], cat_matrix[int(3000000/n):], df_labels['gender'].values, model_class, 2)
    train_score_list.append(val_score)
    test_score_list.append(test_score)
    
    val_score, test_score = \
    tfidf_count_cross_validation(cat_matrix[:int(3000000/n)], cat_matrix[int(3000000/n):], df_labels['age'].values, model_class, 10)
    train_score_list.append(val_score)
    test_score_list.append(test_score)

2020-07-23 09:30:27,776 - INFO - start: 0
2020-07-23 09:30:27,781 - INFO - fold:0
/home/huangweilin/anaconda3/envs/fjw/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)
2020-07-23 09:30:27,866 - INFO - fold:1
2020-07-23 09:30:27,874 - INFO - fold:2
2020-07-23 09:30:27,879 - INFO - fold:3
2020-07-23 09:30:27,884 - INFO - fold:4
2020-07-23 09:30:27,890 - INFO - fold:0
2020-07-23 09:30:27,920 - INFO - fold:1
2020-07-23 09:30:27,944 - INFO - fold:2
2020-07-23 09:30:27,966 - INFO - fold:3
2020-07-23 09:30:27,986 - INFO - fold:4
2020-07-23 09:30:28,008 - INFO - start: 1
2020-07-23 09:30:28,010 - INFO - fold:0
/home/huangweilin/anaconda3/envs/fjw/lib/python3.6/site-packages/sk

In [19]:
train_stack = np.concatenate(train_score_list, axis = 1)
test_stack = np.concatenate(test_score_list, axis = 1)
se_stack = pd.Series(list(np.concatenate([train_stack, test_stack])))
se_stack.index = mp_id_se['ad_id'].index
pickle.dump(se_stack, open('%s/se_tfidf_stack_new.pickle' % my_var_dir, 'wb'))

In [20]:
pickle.dump(se_stack, open('%s/se_tfidf_stack_new.pickle' % fjw_var_dir, 'wb'))

In [21]:
# for i in range(int(len(train_score_list) / 2)):
#     gender_acc, age_acc = accuracy_score(train_score_list[2*i][:, 0] > 0.5, df_labels['gender'] - 1), \
#     accuracy_score(np.argmax(train_score_list[2*i+1], axis = 1), df_labels['age'] - 1)
#     logging.info('%d : gender : %f, age : %f' % (i, gender_acc, age_acc))

# wv

In [21]:
import torch
import torch.nn.functional as F
import torch.utils.data as Data
from torch.autograd import Variable
import pandas as pd
import numpy as np
import time
from sklearn.metrics import roc_auc_score
from sklearn.feature_selection import mutual_info_classif
import datetime
import pickle
import scipy.sparse as ss
import logging
LOG_FORMAT = "%(asctime)s - %(levelname)s - %(message)s"
logging.basicConfig(level=logging.INFO, format=LOG_FORMAT)
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
os.environ['NUMEXPR_MAX_THREADS'] = '32'
# import seaborn as sns

import IPython.display as ipd
import copy
import random
from gensim.models.word2vec import Word2Vec 
import lightgbm as lgb
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold,KFold

In [22]:
window = 175
def generate_wv(name):
    se_user_other = pickle.load(open('%s/se_user_%s.pickle' % (my_var_dir, name), 'rb'))
    
    sentences = []
    for a_user in se_user_other:
        sentence = []
        for item_id in a_user:
            sentence.append(str(item_id))
        sentences.append(sentence)

    wv_model= Word2Vec(sentences, min_count=1, size = mp_column_nembedding[name], window = window, sg = 1, iter = 5, workers = 16)
    
    return wv_model

## single 200 100 dim

In [22]:
mp_column_nembedding = {
    'creative_id' : 200, 
    'ad_id' : 200, 
    'product_id' : 100, 
    'product_category' : 100,
    'advertiser_id' : 200,
    'industry' : 100
}
used_col = ['creative_id', 'ad_id', 'product_id', 'product_category', 'advertiser_id', 'industry']
for col in used_col:
    logging.info('start %s wv' % col)
    wv_model = generate_wv(col)
    pickle.dump(wv_model, open('%s/wv_%s_sg_window%d_dim%d.pickle' % (var_dir, col, window, mp_column_nembedding[col]), 'wb'))
    logging.info('finish %s wv' % col)

2020-07-22 20:17:01,507 - INFO - start creative_id wv
2020-07-22 20:17:01,516 - INFO - collecting all words and their counts
2020-07-22 20:17:01,516 - INFO - PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-07-22 20:17:01,521 - INFO - collected 9984 word types from a corpus of 13273 raw words and 400 sentences
2020-07-22 20:17:01,522 - INFO - Loading a fresh vocabulary
2020-07-22 20:17:01,538 - INFO - effective_min_count=1 retains 9984 unique words (100% of original 9984, drops 0)
2020-07-22 20:17:01,539 - INFO - effective_min_count=1 leaves 13273 word corpus (100% of original 13273, drops 0)
2020-07-22 20:17:01,569 - INFO - deleting the raw counts dictionary of 9984 items
2020-07-22 20:17:01,569 - INFO - sample=0.001 downsamples 5 most-common words
2020-07-22 20:17:01,570 - INFO - downsampling leaves estimated 13247 word corpus (99.8% of prior 13273)
2020-07-22 20:17:01,586 - INFO - estimated required memory for 9984 words and 200 dimensions: 20966400 bytes
2020-

2020-07-22 20:17:06,595 - INFO - worker thread finished; awaiting finish of 8 more threads
2020-07-22 20:17:06,596 - INFO - worker thread finished; awaiting finish of 7 more threads
2020-07-22 20:17:06,597 - INFO - worker thread finished; awaiting finish of 6 more threads
2020-07-22 20:17:06,597 - INFO - worker thread finished; awaiting finish of 5 more threads
2020-07-22 20:17:06,598 - INFO - worker thread finished; awaiting finish of 4 more threads
2020-07-22 20:17:06,599 - INFO - worker thread finished; awaiting finish of 3 more threads
2020-07-22 20:17:06,600 - INFO - worker thread finished; awaiting finish of 2 more threads
2020-07-22 20:17:06,879 - INFO - worker thread finished; awaiting finish of 1 more threads
2020-07-22 20:17:07,222 - INFO - worker thread finished; awaiting finish of 0 more threads
2020-07-22 20:17:07,223 - INFO - EPOCH - 5 : training on 13273 raw words (13242 effective words) took 0.6s, 20815 effective words/s
2020-07-22 20:17:07,224 - INFO - training on a 66

2020-07-22 20:17:11,389 - INFO - worker thread finished; awaiting finish of 4 more threads
2020-07-22 20:17:11,391 - INFO - worker thread finished; awaiting finish of 3 more threads
2020-07-22 20:17:11,392 - INFO - worker thread finished; awaiting finish of 2 more threads
2020-07-22 20:17:11,623 - INFO - worker thread finished; awaiting finish of 1 more threads
2020-07-22 20:17:11,970 - INFO - worker thread finished; awaiting finish of 0 more threads
2020-07-22 20:17:11,971 - INFO - EPOCH - 4 : training on 13273 raw words (13242 effective words) took 0.6s, 21896 effective words/s
2020-07-22 20:17:11,985 - INFO - worker thread finished; awaiting finish of 15 more threads
2020-07-22 20:17:11,986 - INFO - worker thread finished; awaiting finish of 14 more threads
2020-07-22 20:17:11,992 - INFO - worker thread finished; awaiting finish of 13 more threads
2020-07-22 20:17:11,993 - INFO - worker thread finished; awaiting finish of 12 more threads
2020-07-22 20:17:11,994 - INFO - worker threa

2020-07-22 20:17:13,369 - INFO - worker thread finished; awaiting finish of 0 more threads
2020-07-22 20:17:13,370 - INFO - EPOCH - 3 : training on 13273 raw words (5726 effective words) took 0.2s, 34168 effective words/s
2020-07-22 20:17:13,381 - INFO - worker thread finished; awaiting finish of 15 more threads
2020-07-22 20:17:13,386 - INFO - worker thread finished; awaiting finish of 14 more threads
2020-07-22 20:17:13,387 - INFO - worker thread finished; awaiting finish of 13 more threads
2020-07-22 20:17:13,389 - INFO - worker thread finished; awaiting finish of 12 more threads
2020-07-22 20:17:13,390 - INFO - worker thread finished; awaiting finish of 11 more threads
2020-07-22 20:17:13,391 - INFO - worker thread finished; awaiting finish of 10 more threads
2020-07-22 20:17:13,391 - INFO - worker thread finished; awaiting finish of 9 more threads
2020-07-22 20:17:13,392 - INFO - worker thread finished; awaiting finish of 8 more threads
2020-07-22 20:17:13,393 - INFO - worker thre

2020-07-22 20:17:13,829 - INFO - worker thread finished; awaiting finish of 13 more threads
2020-07-22 20:17:13,830 - INFO - worker thread finished; awaiting finish of 12 more threads
2020-07-22 20:17:13,831 - INFO - worker thread finished; awaiting finish of 11 more threads
2020-07-22 20:17:13,832 - INFO - worker thread finished; awaiting finish of 10 more threads
2020-07-22 20:17:13,833 - INFO - worker thread finished; awaiting finish of 9 more threads
2020-07-22 20:17:13,834 - INFO - worker thread finished; awaiting finish of 8 more threads
2020-07-22 20:17:13,835 - INFO - worker thread finished; awaiting finish of 7 more threads
2020-07-22 20:17:13,836 - INFO - worker thread finished; awaiting finish of 6 more threads
2020-07-22 20:17:13,836 - INFO - worker thread finished; awaiting finish of 5 more threads
2020-07-22 20:17:13,837 - INFO - worker thread finished; awaiting finish of 4 more threads
2020-07-22 20:17:13,838 - INFO - worker thread finished; awaiting finish of 3 more thr

2020-07-22 20:17:15,258 - INFO - worker thread finished; awaiting finish of 9 more threads
2020-07-22 20:17:15,259 - INFO - worker thread finished; awaiting finish of 8 more threads
2020-07-22 20:17:15,260 - INFO - worker thread finished; awaiting finish of 7 more threads
2020-07-22 20:17:15,261 - INFO - worker thread finished; awaiting finish of 6 more threads
2020-07-22 20:17:15,263 - INFO - worker thread finished; awaiting finish of 5 more threads
2020-07-22 20:17:15,263 - INFO - worker thread finished; awaiting finish of 4 more threads
2020-07-22 20:17:15,264 - INFO - worker thread finished; awaiting finish of 3 more threads
2020-07-22 20:17:15,264 - INFO - worker thread finished; awaiting finish of 2 more threads
2020-07-22 20:17:15,479 - INFO - worker thread finished; awaiting finish of 1 more threads
2020-07-22 20:17:15,737 - INFO - worker thread finished; awaiting finish of 0 more threads
2020-07-22 20:17:15,738 - INFO - EPOCH - 2 : training on 13273 raw words (11377 effective 

2020-07-22 20:17:17,421 - INFO - worker thread finished; awaiting finish of 5 more threads
2020-07-22 20:17:17,422 - INFO - worker thread finished; awaiting finish of 4 more threads
2020-07-22 20:17:17,423 - INFO - worker thread finished; awaiting finish of 3 more threads
2020-07-22 20:17:17,424 - INFO - worker thread finished; awaiting finish of 2 more threads
2020-07-22 20:17:17,458 - INFO - worker thread finished; awaiting finish of 1 more threads
2020-07-22 20:17:17,478 - INFO - worker thread finished; awaiting finish of 0 more threads
2020-07-22 20:17:17,479 - INFO - EPOCH - 1 : training on 13273 raw words (4437 effective words) took 0.1s, 59507 effective words/s
2020-07-22 20:17:17,491 - INFO - worker thread finished; awaiting finish of 15 more threads
2020-07-22 20:17:17,494 - INFO - worker thread finished; awaiting finish of 14 more threads
2020-07-22 20:17:17,495 - INFO - worker thread finished; awaiting finish of 13 more threads
2020-07-22 20:17:17,495 - INFO - worker thread 

In [29]:
mp_column_nembedding = {
    'creative_id' : 100, 
    'ad_id' : 100, 
    'advertiser_id' : 100,
}
used_col = ['creative_id', 'ad_id', 'advertiser_id',]
for col in used_col:
    logging.info('start %s wv' % col)
    wv_model = generate_wv(col)
    pickle.dump(wv_model, open('%s/wv_%s_sg_window%d_dim%d.pickle' % (var_dir, col, window, mp_column_nembedding[col]), 'wb'))
    logging.info('finish %s wv' % col)

2020-07-23 09:10:07,560 - INFO - start creative_id wv
2020-07-23 09:10:07,747 - INFO - collecting all words and their counts
2020-07-23 09:10:07,748 - INFO - PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-07-23 09:10:07,754 - INFO - collected 9984 word types from a corpus of 13273 raw words and 400 sentences
2020-07-23 09:10:07,755 - INFO - Loading a fresh vocabulary
2020-07-23 09:10:07,796 - INFO - effective_min_count=1 retains 9984 unique words (100% of original 9984, drops 0)
2020-07-23 09:10:07,805 - INFO - effective_min_count=1 leaves 13273 word corpus (100% of original 13273, drops 0)
2020-07-23 09:10:07,840 - INFO - deleting the raw counts dictionary of 9984 items
2020-07-23 09:10:07,841 - INFO - sample=0.001 downsamples 5 most-common words
2020-07-23 09:10:07,842 - INFO - downsampling leaves estimated 13247 word corpus (99.8% of prior 13273)
2020-07-23 09:10:07,858 - INFO - estimated required memory for 9984 words and 100 dimensions: 12979200 bytes
2020-

2020-07-23 09:10:12,554 - INFO - worker thread finished; awaiting finish of 8 more threads
2020-07-23 09:10:12,555 - INFO - worker thread finished; awaiting finish of 7 more threads
2020-07-23 09:10:12,556 - INFO - worker thread finished; awaiting finish of 6 more threads
2020-07-23 09:10:12,556 - INFO - worker thread finished; awaiting finish of 5 more threads
2020-07-23 09:10:12,557 - INFO - worker thread finished; awaiting finish of 4 more threads
2020-07-23 09:10:12,558 - INFO - worker thread finished; awaiting finish of 3 more threads
2020-07-23 09:10:12,559 - INFO - worker thread finished; awaiting finish of 2 more threads
2020-07-23 09:10:12,776 - INFO - worker thread finished; awaiting finish of 1 more threads
2020-07-23 09:10:13,167 - INFO - worker thread finished; awaiting finish of 0 more threads
2020-07-23 09:10:13,168 - INFO - EPOCH - 5 : training on 13273 raw words (13242 effective words) took 0.6s, 21170 effective words/s
2020-07-23 09:10:13,170 - INFO - training on a 66

2020-07-23 09:10:17,073 - INFO - worker thread finished; awaiting finish of 4 more threads
2020-07-23 09:10:17,073 - INFO - worker thread finished; awaiting finish of 3 more threads
2020-07-23 09:10:17,074 - INFO - worker thread finished; awaiting finish of 2 more threads
2020-07-23 09:10:17,261 - INFO - worker thread finished; awaiting finish of 1 more threads
2020-07-23 09:10:17,560 - INFO - worker thread finished; awaiting finish of 0 more threads
2020-07-23 09:10:17,561 - INFO - EPOCH - 4 : training on 13273 raw words (13242 effective words) took 0.5s, 26185 effective words/s
2020-07-23 09:10:17,578 - INFO - worker thread finished; awaiting finish of 15 more threads
2020-07-23 09:10:17,582 - INFO - worker thread finished; awaiting finish of 14 more threads
2020-07-23 09:10:17,583 - INFO - worker thread finished; awaiting finish of 13 more threads
2020-07-23 09:10:17,584 - INFO - worker thread finished; awaiting finish of 12 more threads
2020-07-23 09:10:17,585 - INFO - worker threa

2020-07-23 09:10:20,255 - INFO - worker thread finished; awaiting finish of 0 more threads
2020-07-23 09:10:20,256 - INFO - EPOCH - 3 : training on 13273 raw words (11360 effective words) took 0.4s, 26996 effective words/s
2020-07-23 09:10:20,274 - INFO - worker thread finished; awaiting finish of 15 more threads
2020-07-23 09:10:20,277 - INFO - worker thread finished; awaiting finish of 14 more threads
2020-07-23 09:10:20,278 - INFO - worker thread finished; awaiting finish of 13 more threads
2020-07-23 09:10:20,279 - INFO - worker thread finished; awaiting finish of 12 more threads
2020-07-23 09:10:20,280 - INFO - worker thread finished; awaiting finish of 11 more threads
2020-07-23 09:10:20,281 - INFO - worker thread finished; awaiting finish of 10 more threads
2020-07-23 09:10:20,281 - INFO - worker thread finished; awaiting finish of 9 more threads
2020-07-23 09:10:20,282 - INFO - worker thread finished; awaiting finish of 8 more threads
2020-07-23 09:10:20,282 - INFO - worker thr

## single 50 dim

In [23]:
mp_column_nembedding = {
    'creative_id' : 50, 
    'ad_id' : 50, 
    'product_id' : 50, 
    'product_category' : 50,
    'advertiser_id' : 50,
    'industry' : 50
}
used_col = ['creative_id', 'ad_id', 'product_id', 'product_category', 'advertiser_id', 'industry']
for col in used_col:
    logging.info('start %s wv' % col)
    wv_model = generate_wv(col)
    pickle.dump(wv_model, open('%s/wv_%s_sg_window%d_dim%d.pickle' % (var_dir, col, window, mp_column_nembedding[col]), 'wb'))
    logging.info('finish %s wv' % col)

2020-07-22 20:17:17,842 - INFO - start creative_id wv
2020-07-22 20:17:17,847 - WARNING - consider setting layer size to a multiple of 4 for greater performance
2020-07-22 20:17:17,848 - INFO - collecting all words and their counts
2020-07-22 20:17:17,849 - INFO - PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-07-22 20:17:17,853 - INFO - collected 9984 word types from a corpus of 13273 raw words and 400 sentences
2020-07-22 20:17:17,854 - INFO - Loading a fresh vocabulary
2020-07-22 20:17:17,869 - INFO - effective_min_count=1 retains 9984 unique words (100% of original 9984, drops 0)
2020-07-22 20:17:17,870 - INFO - effective_min_count=1 leaves 13273 word corpus (100% of original 13273, drops 0)
2020-07-22 20:17:17,900 - INFO - deleting the raw counts dictionary of 9984 items
2020-07-22 20:17:17,901 - INFO - sample=0.001 downsamples 5 most-common words
2020-07-22 20:17:17,902 - INFO - downsampling leaves estimated 13247 word corpus (99.8% of prior 13273)
2020-07

2020-07-22 20:17:22,179 - INFO - worker thread finished; awaiting finish of 9 more threads
2020-07-22 20:17:22,179 - INFO - worker thread finished; awaiting finish of 8 more threads
2020-07-22 20:17:22,180 - INFO - worker thread finished; awaiting finish of 7 more threads
2020-07-22 20:17:22,181 - INFO - worker thread finished; awaiting finish of 6 more threads
2020-07-22 20:17:22,184 - INFO - worker thread finished; awaiting finish of 5 more threads
2020-07-22 20:17:22,184 - INFO - worker thread finished; awaiting finish of 4 more threads
2020-07-22 20:17:22,185 - INFO - worker thread finished; awaiting finish of 3 more threads
2020-07-22 20:17:22,185 - INFO - worker thread finished; awaiting finish of 2 more threads
2020-07-22 20:17:22,427 - INFO - worker thread finished; awaiting finish of 1 more threads
2020-07-22 20:17:22,706 - INFO - worker thread finished; awaiting finish of 0 more threads
2020-07-22 20:17:22,707 - INFO - EPOCH - 5 : training on 13273 raw words (13242 effective 

2020-07-22 20:17:26,562 - INFO - worker thread finished; awaiting finish of 6 more threads
2020-07-22 20:17:26,563 - INFO - worker thread finished; awaiting finish of 5 more threads
2020-07-22 20:17:26,564 - INFO - worker thread finished; awaiting finish of 4 more threads
2020-07-22 20:17:26,565 - INFO - worker thread finished; awaiting finish of 3 more threads
2020-07-22 20:17:26,566 - INFO - worker thread finished; awaiting finish of 2 more threads
2020-07-22 20:17:26,818 - INFO - worker thread finished; awaiting finish of 1 more threads
2020-07-22 20:17:27,108 - INFO - worker thread finished; awaiting finish of 0 more threads
2020-07-22 20:17:27,110 - INFO - EPOCH - 4 : training on 13273 raw words (13242 effective words) took 0.6s, 23338 effective words/s
2020-07-22 20:17:27,123 - INFO - worker thread finished; awaiting finish of 15 more threads
2020-07-22 20:17:27,131 - INFO - worker thread finished; awaiting finish of 14 more threads
2020-07-22 20:17:27,132 - INFO - worker thread 

2020-07-22 20:17:28,307 - INFO - worker thread finished; awaiting finish of 3 more threads
2020-07-22 20:17:28,308 - INFO - worker thread finished; awaiting finish of 2 more threads
2020-07-22 20:17:28,340 - INFO - worker thread finished; awaiting finish of 1 more threads
2020-07-22 20:17:28,400 - INFO - worker thread finished; awaiting finish of 0 more threads
2020-07-22 20:17:28,400 - INFO - EPOCH - 3 : training on 13273 raw words (5726 effective words) took 0.1s, 50790 effective words/s
2020-07-22 20:17:28,416 - INFO - worker thread finished; awaiting finish of 15 more threads
2020-07-22 20:17:28,419 - INFO - worker thread finished; awaiting finish of 14 more threads
2020-07-22 20:17:28,420 - INFO - worker thread finished; awaiting finish of 13 more threads
2020-07-22 20:17:28,421 - INFO - worker thread finished; awaiting finish of 12 more threads
2020-07-22 20:17:28,422 - INFO - worker thread finished; awaiting finish of 11 more threads
2020-07-22 20:17:28,423 - INFO - worker threa

2020-07-22 20:17:28,783 - INFO - worker thread finished; awaiting finish of 0 more threads
2020-07-22 20:17:28,784 - INFO - EPOCH - 2 : training on 13273 raw words (1224 effective words) took 0.0s, 58432 effective words/s
2020-07-22 20:17:28,799 - INFO - worker thread finished; awaiting finish of 15 more threads
2020-07-22 20:17:28,800 - INFO - worker thread finished; awaiting finish of 14 more threads
2020-07-22 20:17:28,800 - INFO - worker thread finished; awaiting finish of 13 more threads
2020-07-22 20:17:28,801 - INFO - worker thread finished; awaiting finish of 12 more threads
2020-07-22 20:17:28,802 - INFO - worker thread finished; awaiting finish of 11 more threads
2020-07-22 20:17:28,803 - INFO - worker thread finished; awaiting finish of 10 more threads
2020-07-22 20:17:28,804 - INFO - worker thread finished; awaiting finish of 9 more threads
2020-07-22 20:17:28,804 - INFO - worker thread finished; awaiting finish of 8 more threads
2020-07-22 20:17:28,805 - INFO - worker thre

2020-07-22 20:17:30,130 - INFO - worker thread finished; awaiting finish of 15 more threads
2020-07-22 20:17:30,134 - INFO - worker thread finished; awaiting finish of 14 more threads
2020-07-22 20:17:30,135 - INFO - worker thread finished; awaiting finish of 13 more threads
2020-07-22 20:17:30,136 - INFO - worker thread finished; awaiting finish of 12 more threads
2020-07-22 20:17:30,137 - INFO - worker thread finished; awaiting finish of 11 more threads
2020-07-22 20:17:30,138 - INFO - worker thread finished; awaiting finish of 10 more threads
2020-07-22 20:17:30,139 - INFO - worker thread finished; awaiting finish of 9 more threads
2020-07-22 20:17:30,140 - INFO - worker thread finished; awaiting finish of 8 more threads
2020-07-22 20:17:30,141 - INFO - worker thread finished; awaiting finish of 7 more threads
2020-07-22 20:17:30,142 - INFO - worker thread finished; awaiting finish of 6 more threads
2020-07-22 20:17:30,142 - INFO - worker thread finished; awaiting finish of 5 more t

2020-07-22 20:17:31,862 - INFO - worker thread finished; awaiting finish of 12 more threads
2020-07-22 20:17:31,863 - INFO - worker thread finished; awaiting finish of 11 more threads
2020-07-22 20:17:31,863 - INFO - worker thread finished; awaiting finish of 10 more threads
2020-07-22 20:17:31,864 - INFO - worker thread finished; awaiting finish of 9 more threads
2020-07-22 20:17:31,865 - INFO - worker thread finished; awaiting finish of 8 more threads
2020-07-22 20:17:31,866 - INFO - worker thread finished; awaiting finish of 7 more threads
2020-07-22 20:17:31,866 - INFO - worker thread finished; awaiting finish of 6 more threads
2020-07-22 20:17:31,867 - INFO - worker thread finished; awaiting finish of 5 more threads
2020-07-22 20:17:31,868 - INFO - worker thread finished; awaiting finish of 4 more threads
2020-07-22 20:17:31,869 - INFO - worker thread finished; awaiting finish of 3 more threads
2020-07-22 20:17:31,869 - INFO - worker thread finished; awaiting finish of 2 more thre

## epoch 16

In [23]:
window = 175
def generate_wv_epoch16(name):
    se_user_other = pickle.load(open('%s/se_user_%s.pickle' % (my_var_dir, name), 'rb'))
    
    sentences = []
    for a_user in se_user_other:
        sentence = []
        for item_id in a_user:
            sentence.append(str(item_id))
        sentences.append(sentence)

    wv_model= Word2Vec(sentences, min_count=1, size = mp_column_nembedding[name], window = window, sg = 1, iter = 16, workers = 16)
    
    return wv_model

mp_column_nembedding = {
    'ad_id' : 200, 
    'advertiser_id' : 200,
}
used_col = [ 'ad_id',   'advertiser_id']
for col in used_col:
    logging.info('start %s wv' % col)
    wv_model = generate_wv_epoch16(col)
    pickle.dump(wv_model, open('%s/wv_%s_sg_window%d_dim%d_epoch16.pickle' % (var_dir, col, window, mp_column_nembedding[col]), 'wb'))
    logging.info('finish %s wv' % col)

2020-07-23 09:35:01,622 - INFO - start ad_id wv
2020-07-23 09:35:01,635 - INFO - collecting all words and their counts
2020-07-23 09:35:01,636 - INFO - PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-07-23 09:35:01,645 - INFO - collected 9909 word types from a corpus of 13273 raw words and 400 sentences
2020-07-23 09:35:01,645 - INFO - Loading a fresh vocabulary
2020-07-23 09:35:01,680 - INFO - effective_min_count=1 retains 9909 unique words (100% of original 9909, drops 0)
2020-07-23 09:35:01,681 - INFO - effective_min_count=1 leaves 13273 word corpus (100% of original 13273, drops 0)
2020-07-23 09:35:01,729 - INFO - deleting the raw counts dictionary of 9909 items
2020-07-23 09:35:01,731 - INFO - sample=0.001 downsamples 6 most-common words
2020-07-23 09:35:01,732 - INFO - downsampling leaves estimated 13246 word corpus (99.8% of prior 13273)
2020-07-23 09:35:01,754 - INFO - estimated required memory for 9909 words and 200 dimensions: 20808900 bytes
2020-07-23 

2020-07-23 09:35:06,742 - INFO - worker thread finished; awaiting finish of 8 more threads
2020-07-23 09:35:06,743 - INFO - worker thread finished; awaiting finish of 7 more threads
2020-07-23 09:35:06,744 - INFO - worker thread finished; awaiting finish of 6 more threads
2020-07-23 09:35:06,745 - INFO - worker thread finished; awaiting finish of 5 more threads
2020-07-23 09:35:06,745 - INFO - worker thread finished; awaiting finish of 4 more threads
2020-07-23 09:35:06,746 - INFO - worker thread finished; awaiting finish of 3 more threads
2020-07-23 09:35:06,749 - INFO - worker thread finished; awaiting finish of 2 more threads
2020-07-23 09:35:06,979 - INFO - worker thread finished; awaiting finish of 1 more threads
2020-07-23 09:35:07,395 - INFO - worker thread finished; awaiting finish of 0 more threads
2020-07-23 09:35:07,396 - INFO - EPOCH - 5 : training on 13273 raw words (13243 effective words) took 0.7s, 19749 effective words/s
2020-07-23 09:35:07,411 - INFO - worker thread fi

2020-07-23 09:35:10,117 - INFO - worker thread finished; awaiting finish of 5 more threads
2020-07-23 09:35:10,117 - INFO - worker thread finished; awaiting finish of 4 more threads
2020-07-23 09:35:10,118 - INFO - worker thread finished; awaiting finish of 3 more threads
2020-07-23 09:35:10,119 - INFO - worker thread finished; awaiting finish of 2 more threads
2020-07-23 09:35:10,449 - INFO - worker thread finished; awaiting finish of 1 more threads
2020-07-23 09:35:10,828 - INFO - worker thread finished; awaiting finish of 0 more threads
2020-07-23 09:35:10,829 - INFO - EPOCH - 10 : training on 13273 raw words (13251 effective words) took 0.7s, 18026 effective words/s
2020-07-23 09:35:10,844 - INFO - worker thread finished; awaiting finish of 15 more threads
2020-07-23 09:35:10,849 - INFO - worker thread finished; awaiting finish of 14 more threads
2020-07-23 09:35:10,852 - INFO - worker thread finished; awaiting finish of 13 more threads
2020-07-23 09:35:10,853 - INFO - worker threa

2020-07-23 09:35:13,654 - INFO - worker thread finished; awaiting finish of 2 more threads
2020-07-23 09:35:13,866 - INFO - worker thread finished; awaiting finish of 1 more threads
2020-07-23 09:35:14,250 - INFO - worker thread finished; awaiting finish of 0 more threads
2020-07-23 09:35:14,251 - INFO - EPOCH - 15 : training on 13273 raw words (13252 effective words) took 0.6s, 21414 effective words/s
2020-07-23 09:35:14,264 - INFO - worker thread finished; awaiting finish of 15 more threads
2020-07-23 09:35:14,270 - INFO - worker thread finished; awaiting finish of 14 more threads
2020-07-23 09:35:14,272 - INFO - worker thread finished; awaiting finish of 13 more threads
2020-07-23 09:35:14,273 - INFO - worker thread finished; awaiting finish of 12 more threads
2020-07-23 09:35:14,274 - INFO - worker thread finished; awaiting finish of 11 more threads
2020-07-23 09:35:14,275 - INFO - worker thread finished; awaiting finish of 10 more threads
2020-07-23 09:35:14,276 - INFO - worker th

2020-07-23 09:35:17,211 - INFO - worker thread finished; awaiting finish of 15 more threads
2020-07-23 09:35:17,215 - INFO - worker thread finished; awaiting finish of 14 more threads
2020-07-23 09:35:17,219 - INFO - worker thread finished; awaiting finish of 13 more threads
2020-07-23 09:35:17,219 - INFO - worker thread finished; awaiting finish of 12 more threads
2020-07-23 09:35:17,220 - INFO - worker thread finished; awaiting finish of 11 more threads
2020-07-23 09:35:17,221 - INFO - worker thread finished; awaiting finish of 10 more threads
2020-07-23 09:35:17,222 - INFO - worker thread finished; awaiting finish of 9 more threads
2020-07-23 09:35:17,223 - INFO - worker thread finished; awaiting finish of 8 more threads
2020-07-23 09:35:17,223 - INFO - worker thread finished; awaiting finish of 7 more threads
2020-07-23 09:35:17,224 - INFO - worker thread finished; awaiting finish of 6 more threads
2020-07-23 09:35:17,225 - INFO - worker thread finished; awaiting finish of 5 more t

2020-07-23 09:35:19,945 - INFO - worker thread finished; awaiting finish of 12 more threads
2020-07-23 09:35:19,946 - INFO - worker thread finished; awaiting finish of 11 more threads
2020-07-23 09:35:19,947 - INFO - worker thread finished; awaiting finish of 10 more threads
2020-07-23 09:35:19,948 - INFO - worker thread finished; awaiting finish of 9 more threads
2020-07-23 09:35:19,949 - INFO - worker thread finished; awaiting finish of 8 more threads
2020-07-23 09:35:19,950 - INFO - worker thread finished; awaiting finish of 7 more threads
2020-07-23 09:35:19,951 - INFO - worker thread finished; awaiting finish of 6 more threads
2020-07-23 09:35:19,951 - INFO - worker thread finished; awaiting finish of 5 more threads
2020-07-23 09:35:19,952 - INFO - worker thread finished; awaiting finish of 4 more threads
2020-07-23 09:35:19,953 - INFO - worker thread finished; awaiting finish of 3 more threads
2020-07-23 09:35:19,954 - INFO - worker thread finished; awaiting finish of 2 more thre

2020-07-23 09:35:22,480 - INFO - worker thread finished; awaiting finish of 9 more threads
2020-07-23 09:35:22,481 - INFO - worker thread finished; awaiting finish of 8 more threads
2020-07-23 09:35:22,482 - INFO - worker thread finished; awaiting finish of 7 more threads
2020-07-23 09:35:22,483 - INFO - worker thread finished; awaiting finish of 6 more threads
2020-07-23 09:35:22,484 - INFO - worker thread finished; awaiting finish of 5 more threads
2020-07-23 09:35:22,485 - INFO - worker thread finished; awaiting finish of 4 more threads
2020-07-23 09:35:22,487 - INFO - worker thread finished; awaiting finish of 3 more threads
2020-07-23 09:35:22,488 - INFO - worker thread finished; awaiting finish of 2 more threads
2020-07-23 09:35:22,696 - INFO - worker thread finished; awaiting finish of 1 more threads
2020-07-23 09:35:22,952 - INFO - worker thread finished; awaiting finish of 0 more threads
2020-07-23 09:35:22,953 - INFO - EPOCH - 14 : training on 13273 raw words (11368 effective

## cross

In [24]:
window = 175
cross_id_set = set()
def word2vec_cross(se_feature_sequence1, se_feature_sequence2, n_dim):
    sentences = []
    for f1, f2 in zip(se_feature_sequence1, se_feature_sequence2):
        sentence = []
        for item_id1, item_id2, in zip(f1, f2):
            cross_id = "%s_%s"%(str(item_id1), str(item_id2))
            sentence.append(cross_id)
        sentences.append(sentence)
        cross_id_set.add(cross_id)
        
    logging.info('unique id:%d' % len(set(cross_id_set)))
    wv_model= Word2Vec(sentences, min_count=1, size = n_dim, window = window, sg = 1, workers = 24)
    

    return wv_model

def generate_cross(feature1, feature2, n_dim = 50):
    se_feature_sequence1 = mp_id_se[feature1]
    se_feature_sequence2 = mp_id_se[feature2]
    logging.info('start corss %s %s wv' % (feature1, feature2))
    model = word2vec_cross(se_feature_sequence1, se_feature_sequence2, n_dim)
    pickle.dump(model, open('%s/cross_wv_model_%s_%s_sg_window%d_dim%d.pickle' % \
                            (var_dir, feature1, feature2, window, n_dim), 'wb'))
    logging.info('finish corss %s %s wv' % (feature1, feature2))


In [25]:
generate_cross('product_id', 'advertiser_id')
generate_cross('product_id', 'industry')
generate_cross('product_id', 'product_category')
generate_cross('product_category', 'advertiser_id')
generate_cross('product_category', 'industry')
generate_cross('advertiser_id', 'industry')

2020-07-22 20:17:32,345 - INFO - start corss product_id advertiser_id wv
2020-07-22 20:17:32,364 - INFO - unique id:287
2020-07-22 20:17:32,365 - WARNING - consider setting layer size to a multiple of 4 for greater performance
2020-07-22 20:17:32,366 - INFO - collecting all words and their counts
2020-07-22 20:17:32,366 - INFO - PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-07-22 20:17:32,371 - INFO - collected 3807 word types from a corpus of 13273 raw words and 400 sentences
2020-07-22 20:17:32,372 - INFO - Loading a fresh vocabulary
2020-07-22 20:17:32,381 - INFO - effective_min_count=1 retains 3807 unique words (100% of original 3807, drops 0)
2020-07-22 20:17:32,382 - INFO - effective_min_count=1 leaves 13273 word corpus (100% of original 13273, drops 0)
2020-07-22 20:17:32,398 - INFO - deleting the raw counts dictionary of 3807 items
2020-07-22 20:17:32,400 - INFO - sample=0.001 downsamples 43 most-common words
2020-07-22 20:17:32,400 - INFO - downsamplin

2020-07-22 20:17:34,468 - INFO - EPOCH - 3 : training on 13273 raw words (11432 effective words) took 0.4s, 29389 effective words/s
2020-07-22 20:17:34,487 - INFO - worker thread finished; awaiting finish of 23 more threads
2020-07-22 20:17:34,490 - INFO - worker thread finished; awaiting finish of 22 more threads
2020-07-22 20:17:34,492 - INFO - worker thread finished; awaiting finish of 21 more threads
2020-07-22 20:17:34,493 - INFO - worker thread finished; awaiting finish of 20 more threads
2020-07-22 20:17:34,494 - INFO - worker thread finished; awaiting finish of 19 more threads
2020-07-22 20:17:34,495 - INFO - worker thread finished; awaiting finish of 18 more threads
2020-07-22 20:17:34,496 - INFO - worker thread finished; awaiting finish of 17 more threads
2020-07-22 20:17:34,497 - INFO - worker thread finished; awaiting finish of 16 more threads
2020-07-22 20:17:34,497 - INFO - worker thread finished; awaiting finish of 15 more threads
2020-07-22 20:17:34,498 - INFO - worker 

2020-07-22 20:17:35,745 - INFO - worker thread finished; awaiting finish of 4 more threads
2020-07-22 20:17:35,746 - INFO - worker thread finished; awaiting finish of 3 more threads
2020-07-22 20:17:35,747 - INFO - worker thread finished; awaiting finish of 2 more threads
2020-07-22 20:17:35,830 - INFO - worker thread finished; awaiting finish of 1 more threads
2020-07-22 20:17:35,930 - INFO - worker thread finished; awaiting finish of 0 more threads
2020-07-22 20:17:35,931 - INFO - EPOCH - 1 : training on 13273 raw words (8209 effective words) took 0.2s, 38540 effective words/s
2020-07-22 20:17:35,949 - INFO - worker thread finished; awaiting finish of 23 more threads
2020-07-22 20:17:35,952 - INFO - worker thread finished; awaiting finish of 22 more threads
2020-07-22 20:17:35,953 - INFO - worker thread finished; awaiting finish of 21 more threads
2020-07-22 20:17:35,954 - INFO - worker thread finished; awaiting finish of 20 more threads
2020-07-22 20:17:35,955 - INFO - worker thread

2020-07-22 20:17:36,657 - INFO - worker thread finished; awaiting finish of 16 more threads
2020-07-22 20:17:36,658 - INFO - worker thread finished; awaiting finish of 15 more threads
2020-07-22 20:17:36,659 - INFO - worker thread finished; awaiting finish of 14 more threads
2020-07-22 20:17:36,660 - INFO - worker thread finished; awaiting finish of 13 more threads
2020-07-22 20:17:36,661 - INFO - worker thread finished; awaiting finish of 12 more threads
2020-07-22 20:17:36,662 - INFO - worker thread finished; awaiting finish of 11 more threads
2020-07-22 20:17:36,662 - INFO - worker thread finished; awaiting finish of 10 more threads
2020-07-22 20:17:36,665 - INFO - worker thread finished; awaiting finish of 9 more threads
2020-07-22 20:17:36,666 - INFO - worker thread finished; awaiting finish of 8 more threads
2020-07-22 20:17:36,667 - INFO - worker thread finished; awaiting finish of 7 more threads
2020-07-22 20:17:36,667 - INFO - worker thread finished; awaiting finish of 6 more 

2020-07-22 20:17:37,475 - INFO - worker thread finished; awaiting finish of 21 more threads
2020-07-22 20:17:37,476 - INFO - worker thread finished; awaiting finish of 20 more threads
2020-07-22 20:17:37,477 - INFO - worker thread finished; awaiting finish of 19 more threads
2020-07-22 20:17:37,477 - INFO - worker thread finished; awaiting finish of 18 more threads
2020-07-22 20:17:37,478 - INFO - worker thread finished; awaiting finish of 17 more threads
2020-07-22 20:17:37,479 - INFO - worker thread finished; awaiting finish of 16 more threads
2020-07-22 20:17:37,480 - INFO - worker thread finished; awaiting finish of 15 more threads
2020-07-22 20:17:37,481 - INFO - worker thread finished; awaiting finish of 14 more threads
2020-07-22 20:17:37,482 - INFO - worker thread finished; awaiting finish of 13 more threads
2020-07-22 20:17:37,482 - INFO - worker thread finished; awaiting finish of 12 more threads
2020-07-22 20:17:37,483 - INFO - worker thread finished; awaiting finish of 11 m

2020-07-22 20:17:37,939 - INFO - estimated required memory for 3726 words and 50 dimensions: 3353400 bytes
2020-07-22 20:17:37,940 - INFO - resetting layer weights
2020-07-22 20:17:38,717 - INFO - training model with 24 workers on 3726 vocabulary and 50 features, using sg=1 hs=0 sample=0.001 negative=5 window=175
2020-07-22 20:17:38,734 - INFO - worker thread finished; awaiting finish of 23 more threads
2020-07-22 20:17:38,739 - INFO - worker thread finished; awaiting finish of 22 more threads
2020-07-22 20:17:38,741 - INFO - worker thread finished; awaiting finish of 21 more threads
2020-07-22 20:17:38,743 - INFO - worker thread finished; awaiting finish of 20 more threads
2020-07-22 20:17:38,744 - INFO - worker thread finished; awaiting finish of 19 more threads
2020-07-22 20:17:38,746 - INFO - worker thread finished; awaiting finish of 18 more threads
2020-07-22 20:17:38,747 - INFO - worker thread finished; awaiting finish of 17 more threads
2020-07-22 20:17:38,748 - INFO - worker t

2020-07-22 20:17:40,075 - INFO - worker thread finished; awaiting finish of 13 more threads
2020-07-22 20:17:40,076 - INFO - worker thread finished; awaiting finish of 12 more threads
2020-07-22 20:17:40,076 - INFO - worker thread finished; awaiting finish of 11 more threads
2020-07-22 20:17:40,077 - INFO - worker thread finished; awaiting finish of 10 more threads
2020-07-22 20:17:40,078 - INFO - worker thread finished; awaiting finish of 9 more threads
2020-07-22 20:17:40,078 - INFO - worker thread finished; awaiting finish of 8 more threads
2020-07-22 20:17:40,079 - INFO - worker thread finished; awaiting finish of 7 more threads
2020-07-22 20:17:40,080 - INFO - worker thread finished; awaiting finish of 6 more threads
2020-07-22 20:17:40,080 - INFO - worker thread finished; awaiting finish of 5 more threads
2020-07-22 20:17:40,081 - INFO - worker thread finished; awaiting finish of 4 more threads
2020-07-22 20:17:40,082 - INFO - worker thread finished; awaiting finish of 3 more thr

2020-07-22 20:17:41,166 - INFO - worker thread finished; awaiting finish of 18 more threads
2020-07-22 20:17:41,167 - INFO - worker thread finished; awaiting finish of 17 more threads
2020-07-22 20:17:41,168 - INFO - worker thread finished; awaiting finish of 16 more threads
2020-07-22 20:17:41,169 - INFO - worker thread finished; awaiting finish of 15 more threads
2020-07-22 20:17:41,169 - INFO - worker thread finished; awaiting finish of 14 more threads
2020-07-22 20:17:41,170 - INFO - worker thread finished; awaiting finish of 13 more threads
2020-07-22 20:17:41,171 - INFO - worker thread finished; awaiting finish of 12 more threads
2020-07-22 20:17:41,171 - INFO - worker thread finished; awaiting finish of 11 more threads
2020-07-22 20:17:41,174 - INFO - worker thread finished; awaiting finish of 10 more threads
2020-07-22 20:17:41,174 - INFO - worker thread finished; awaiting finish of 9 more threads
2020-07-22 20:17:41,175 - INFO - worker thread finished; awaiting finish of 8 mor

2020-07-22 20:17:41,563 - INFO - worker thread finished; awaiting finish of 4 more threads
2020-07-22 20:17:41,563 - INFO - worker thread finished; awaiting finish of 3 more threads
2020-07-22 20:17:41,564 - INFO - worker thread finished; awaiting finish of 2 more threads
2020-07-22 20:17:41,590 - INFO - worker thread finished; awaiting finish of 1 more threads
2020-07-22 20:17:41,663 - INFO - worker thread finished; awaiting finish of 0 more threads
2020-07-22 20:17:41,664 - INFO - EPOCH - 5 : training on 13273 raw words (6118 effective words) took 0.1s, 48734 effective words/s
2020-07-22 20:17:41,665 - INFO - training on a 66365 raw words (30751 effective words) took 0.7s, 47116 effective words/s
2020-07-22 20:17:41,666 - WARNING - under 10 jobs per worker: consider setting a smaller `batch_words' for smoother alpha decay
2020-07-22 20:17:41,670 - INFO - finish corss product_category industry wv
2020-07-22 20:17:41,670 - INFO - start corss advertiser_id industry wv
2020-07-22 20:17:4

2020-07-22 20:17:43,545 - INFO - worker thread finished; awaiting finish of 9 more threads
2020-07-22 20:17:43,546 - INFO - worker thread finished; awaiting finish of 8 more threads
2020-07-22 20:17:43,546 - INFO - worker thread finished; awaiting finish of 7 more threads
2020-07-22 20:17:43,547 - INFO - worker thread finished; awaiting finish of 6 more threads
2020-07-22 20:17:43,548 - INFO - worker thread finished; awaiting finish of 5 more threads
2020-07-22 20:17:43,548 - INFO - worker thread finished; awaiting finish of 4 more threads
2020-07-22 20:17:43,552 - INFO - worker thread finished; awaiting finish of 3 more threads
2020-07-22 20:17:43,553 - INFO - worker thread finished; awaiting finish of 2 more threads
2020-07-22 20:17:43,734 - INFO - worker thread finished; awaiting finish of 1 more threads
2020-07-22 20:17:44,004 - INFO - worker thread finished; awaiting finish of 0 more threads
2020-07-22 20:17:44,005 - INFO - EPOCH - 3 : training on 13273 raw words (12006 effective 

In [26]:
generate_cross('product_id', 'advertiser_id', 100)
generate_cross('product_id', 'industry', 100)
generate_cross('product_id', 'product_category', 100)
generate_cross('product_category', 'advertiser_id', 100)
generate_cross('product_category', 'industry', 100)
generate_cross('advertiser_id', 'industry', 100)

2020-07-22 20:17:44,856 - INFO - start corss product_id advertiser_id wv
2020-07-22 20:17:44,866 - INFO - unique id:1317
2020-07-22 20:17:44,868 - INFO - collecting all words and their counts
2020-07-22 20:17:44,868 - INFO - PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-07-22 20:17:44,872 - INFO - collected 3807 word types from a corpus of 13273 raw words and 400 sentences
2020-07-22 20:17:44,873 - INFO - Loading a fresh vocabulary
2020-07-22 20:17:44,879 - INFO - effective_min_count=1 retains 3807 unique words (100% of original 3807, drops 0)
2020-07-22 20:17:44,880 - INFO - effective_min_count=1 leaves 13273 word corpus (100% of original 13273, drops 0)
2020-07-22 20:17:44,892 - INFO - deleting the raw counts dictionary of 3807 items
2020-07-22 20:17:44,893 - INFO - sample=0.001 downsamples 43 most-common words
2020-07-22 20:17:44,894 - INFO - downsampling leaves estimated 11447 word corpus (86.2% of prior 13273)
2020-07-22 20:17:44,901 - INFO - estimated req

2020-07-22 20:17:47,036 - INFO - worker thread finished; awaiting finish of 23 more threads
2020-07-22 20:17:47,043 - INFO - worker thread finished; awaiting finish of 22 more threads
2020-07-22 20:17:47,045 - INFO - worker thread finished; awaiting finish of 21 more threads
2020-07-22 20:17:47,046 - INFO - worker thread finished; awaiting finish of 20 more threads
2020-07-22 20:17:47,047 - INFO - worker thread finished; awaiting finish of 19 more threads
2020-07-22 20:17:47,048 - INFO - worker thread finished; awaiting finish of 18 more threads
2020-07-22 20:17:47,049 - INFO - worker thread finished; awaiting finish of 17 more threads
2020-07-22 20:17:47,049 - INFO - worker thread finished; awaiting finish of 16 more threads
2020-07-22 20:17:47,050 - INFO - worker thread finished; awaiting finish of 15 more threads
2020-07-22 20:17:47,051 - INFO - worker thread finished; awaiting finish of 14 more threads
2020-07-22 20:17:47,052 - INFO - worker thread finished; awaiting finish of 13 m

2020-07-22 20:17:48,400 - INFO - worker thread finished; awaiting finish of 1 more threads
2020-07-22 20:17:48,530 - INFO - worker thread finished; awaiting finish of 0 more threads
2020-07-22 20:17:48,531 - INFO - EPOCH - 1 : training on 13273 raw words (8209 effective words) took 0.2s, 34431 effective words/s
2020-07-22 20:17:48,549 - INFO - worker thread finished; awaiting finish of 23 more threads
2020-07-22 20:17:48,554 - INFO - worker thread finished; awaiting finish of 22 more threads
2020-07-22 20:17:48,555 - INFO - worker thread finished; awaiting finish of 21 more threads
2020-07-22 20:17:48,556 - INFO - worker thread finished; awaiting finish of 20 more threads
2020-07-22 20:17:48,557 - INFO - worker thread finished; awaiting finish of 19 more threads
2020-07-22 20:17:48,558 - INFO - worker thread finished; awaiting finish of 18 more threads
2020-07-22 20:17:48,559 - INFO - worker thread finished; awaiting finish of 17 more threads
2020-07-22 20:17:48,560 - INFO - worker thr

2020-07-22 20:17:49,255 - INFO - worker thread finished; awaiting finish of 13 more threads
2020-07-22 20:17:49,256 - INFO - worker thread finished; awaiting finish of 12 more threads
2020-07-22 20:17:49,257 - INFO - worker thread finished; awaiting finish of 11 more threads
2020-07-22 20:17:49,258 - INFO - worker thread finished; awaiting finish of 10 more threads
2020-07-22 20:17:49,259 - INFO - worker thread finished; awaiting finish of 9 more threads
2020-07-22 20:17:49,259 - INFO - worker thread finished; awaiting finish of 8 more threads
2020-07-22 20:17:49,260 - INFO - worker thread finished; awaiting finish of 7 more threads
2020-07-22 20:17:49,261 - INFO - worker thread finished; awaiting finish of 6 more threads
2020-07-22 20:17:49,261 - INFO - worker thread finished; awaiting finish of 5 more threads
2020-07-22 20:17:49,262 - INFO - worker thread finished; awaiting finish of 4 more threads
2020-07-22 20:17:49,262 - INFO - worker thread finished; awaiting finish of 3 more thr

2020-07-22 20:17:50,167 - INFO - worker thread finished; awaiting finish of 17 more threads
2020-07-22 20:17:50,169 - INFO - worker thread finished; awaiting finish of 16 more threads
2020-07-22 20:17:50,170 - INFO - worker thread finished; awaiting finish of 15 more threads
2020-07-22 20:17:50,171 - INFO - worker thread finished; awaiting finish of 14 more threads
2020-07-22 20:17:50,172 - INFO - worker thread finished; awaiting finish of 13 more threads
2020-07-22 20:17:50,175 - INFO - worker thread finished; awaiting finish of 12 more threads
2020-07-22 20:17:50,176 - INFO - worker thread finished; awaiting finish of 11 more threads
2020-07-22 20:17:50,176 - INFO - worker thread finished; awaiting finish of 10 more threads
2020-07-22 20:17:50,177 - INFO - worker thread finished; awaiting finish of 9 more threads
2020-07-22 20:17:50,178 - INFO - worker thread finished; awaiting finish of 8 more threads
2020-07-22 20:17:50,178 - INFO - worker thread finished; awaiting finish of 7 more

2020-07-22 20:17:51,467 - INFO - worker thread finished; awaiting finish of 21 more threads
2020-07-22 20:17:51,468 - INFO - worker thread finished; awaiting finish of 20 more threads
2020-07-22 20:17:51,469 - INFO - worker thread finished; awaiting finish of 19 more threads
2020-07-22 20:17:51,470 - INFO - worker thread finished; awaiting finish of 18 more threads
2020-07-22 20:17:51,471 - INFO - worker thread finished; awaiting finish of 17 more threads
2020-07-22 20:17:51,471 - INFO - worker thread finished; awaiting finish of 16 more threads
2020-07-22 20:17:51,472 - INFO - worker thread finished; awaiting finish of 15 more threads
2020-07-22 20:17:51,474 - INFO - worker thread finished; awaiting finish of 14 more threads
2020-07-22 20:17:51,475 - INFO - worker thread finished; awaiting finish of 13 more threads
2020-07-22 20:17:51,475 - INFO - worker thread finished; awaiting finish of 12 more threads
2020-07-22 20:17:51,476 - INFO - worker thread finished; awaiting finish of 11 m

2020-07-22 20:17:52,795 - INFO - worker thread finished; awaiting finish of 7 more threads
2020-07-22 20:17:52,797 - INFO - worker thread finished; awaiting finish of 6 more threads
2020-07-22 20:17:52,798 - INFO - worker thread finished; awaiting finish of 5 more threads
2020-07-22 20:17:52,798 - INFO - worker thread finished; awaiting finish of 4 more threads
2020-07-22 20:17:52,799 - INFO - worker thread finished; awaiting finish of 3 more threads
2020-07-22 20:17:52,800 - INFO - worker thread finished; awaiting finish of 2 more threads
2020-07-22 20:17:52,985 - INFO - worker thread finished; awaiting finish of 1 more threads
2020-07-22 20:17:53,190 - INFO - worker thread finished; awaiting finish of 0 more threads
2020-07-22 20:17:53,191 - INFO - EPOCH - 4 : training on 13273 raw words (11517 effective words) took 0.4s, 27236 effective words/s
2020-07-22 20:17:53,211 - INFO - worker thread finished; awaiting finish of 23 more threads
2020-07-22 20:17:53,214 - INFO - worker thread f

2020-07-22 20:17:53,911 - INFO - worker thread finished; awaiting finish of 11 more threads
2020-07-22 20:17:53,911 - INFO - worker thread finished; awaiting finish of 10 more threads
2020-07-22 20:17:53,913 - INFO - worker thread finished; awaiting finish of 9 more threads
2020-07-22 20:17:53,913 - INFO - worker thread finished; awaiting finish of 8 more threads
2020-07-22 20:17:53,914 - INFO - worker thread finished; awaiting finish of 7 more threads
2020-07-22 20:17:53,915 - INFO - worker thread finished; awaiting finish of 6 more threads
2020-07-22 20:17:53,917 - INFO - worker thread finished; awaiting finish of 5 more threads
2020-07-22 20:17:53,917 - INFO - worker thread finished; awaiting finish of 4 more threads
2020-07-22 20:17:53,918 - INFO - worker thread finished; awaiting finish of 3 more threads
2020-07-22 20:17:53,919 - INFO - worker thread finished; awaiting finish of 2 more threads
2020-07-22 20:17:53,962 - INFO - worker thread finished; awaiting finish of 1 more threa

2020-07-22 20:17:54,468 - INFO - training on a 66365 raw words (30751 effective words) took 0.8s, 40737 effective words/s
2020-07-22 20:17:54,469 - WARNING - under 10 jobs per worker: consider setting a smaller `batch_words' for smoother alpha decay
2020-07-22 20:17:54,474 - INFO - finish corss product_category industry wv
2020-07-22 20:17:54,475 - INFO - start corss advertiser_id industry wv
2020-07-22 20:17:54,490 - INFO - unique id:1317
2020-07-22 20:17:54,491 - INFO - collecting all words and their counts
2020-07-22 20:17:54,492 - INFO - PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-07-22 20:17:54,495 - INFO - collected 4116 word types from a corpus of 13273 raw words and 400 sentences
2020-07-22 20:17:54,496 - INFO - Loading a fresh vocabulary
2020-07-22 20:17:54,503 - INFO - effective_min_count=1 retains 4116 unique words (100% of original 4116, drops 0)
2020-07-22 20:17:54,504 - INFO - effective_min_count=1 leaves 13273 word corpus (100% of original 1327

2020-07-22 20:17:56,535 - INFO - worker thread finished; awaiting finish of 1 more threads
2020-07-22 20:17:56,836 - INFO - worker thread finished; awaiting finish of 0 more threads
2020-07-22 20:17:56,837 - INFO - EPOCH - 3 : training on 13273 raw words (12006 effective words) took 0.5s, 26279 effective words/s
2020-07-22 20:17:56,854 - INFO - worker thread finished; awaiting finish of 23 more threads
2020-07-22 20:17:56,857 - INFO - worker thread finished; awaiting finish of 22 more threads
2020-07-22 20:17:56,858 - INFO - worker thread finished; awaiting finish of 21 more threads
2020-07-22 20:17:56,859 - INFO - worker thread finished; awaiting finish of 20 more threads
2020-07-22 20:17:56,860 - INFO - worker thread finished; awaiting finish of 19 more threads
2020-07-22 20:17:56,861 - INFO - worker thread finished; awaiting finish of 18 more threads
2020-07-22 20:17:56,862 - INFO - worker thread finished; awaiting finish of 17 more threads
2020-07-22 20:17:56,863 - INFO - worker th